In [1]:
from typing import List
from datetime import date, timedelta
import secrets

from sqlalchemy.orm import Session

from sqlalchemy import asc, or_, and_, func, not_
from sqlalchemy.sql import text

from sql_app import crud, models, schemas
from sql_app.database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

db = SessionLocal()

date_from = date.today()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation,isouter=True).\
        filter(or_(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date),\
               models.CostumeItem.reservation_id == None))
    all_costumes = db.query(models.CostumeItem)
    items = db.query(models.CostumeItem).filter_by(reservation_id=None).order_by(models.CostumeItem.id.asc())
    
finally:
    db.close()


In [21]:
db = SessionLocal()
user_id=5
res_id = 1

try:
    reservation = db.query(models.Reservation).filter_by(id=res_id, client_id=user_id).first()
    print(f'{[{"id":item.id, "model":item.model_id, "loc":item.location_id} for item in reservation.items]}')
    print(len(reservation.items))
    for item in reservation.items:
        item.reservation_id=None
    db.commit()
finally:
    db.close()

[{'id': 1, 'model': 1, 'loc': 1}, {'id': 7, 'model': 7, 'loc': 1}, {'id': 8, 'model': 3, 'loc': 2}, {'id': 9, 'model': 7, 'loc': 2}, {'id': 10, 'model': 7, 'loc': 2}, {'id': 11, 'model': 7, 'loc': 2}, {'id': 12, 'model': 7, 'loc': 2}, {'id': 14, 'model': 3, 'loc': 1}, {'id': 15, 'model': 3, 'loc': 1}, {'id': 16, 'model': 3, 'loc': 1}]
10


In [20]:
db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).\
        filter(models.CostumeItem.rental_id == None)
finally:
    db.close()

In [22]:
str(available_costumes)

'SELECT "Costume_items".costume_id AS "Costume_items_costume_id", "Costume_items".model_id AS "Costume_items_model_id", "Costume_items".location_id AS "Costume_items_location_id", "Costume_items".rental_id AS "Costume_items_rental_id", "Costume_items".reservation_id AS "Costume_items_reservation_id" \nFROM "Costume_items" \nWHERE "Costume_items".rental_id IS NULL'

In [21]:
available_costumes.all()

In [30]:
costumes= [
    {
      "model_id": 4,
      "quantity": 2
    },
    {
      "model_id": 7,
      "quantity": 1
    },
    {
      "model_id": 1,
      "quantity": 7
    }
  ]

count = {4:2, 7:1}

In [33]:
for id_, quan in zip(count, count.values()):
    print(f'id: {id_}, quantity:{quan}')

id: 4, quantity:2
id: 7, quantity:1


In [16]:
db = SessionLocal()
try:
    res_db = models.Reservation(id=4,client_id=1, pick_up_location_id=1,date=date.today(), pick_up_date=date.today()+timedelta(days=10),return_date=date.today()+timedelta(days=20), valid=True)
    db.add(res_db)
    db.commit()
finally:
    db.close()

In [22]:
db = SessionLocal()
try:
    res_db = db.query(models.Reservation).get(2)
    print(res_db.__dict__)
finally:
    db.close()

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7fb82c4879d0>, 'pick_up_location_id': 2, 'valid': False, 'pick_up_date': datetime.datetime(2020, 6, 20, 0, 0), 'id': 2, 'return_date': datetime.datetime(2020, 7, 6, 0, 0), 'client_id': 2, 'date': datetime.datetime(2020, 6, 16, 0, 0)}


In [31]:
import base64
username = "test"
password = "admin1"
base64.b64decode(username+':'+password)

Error: Incorrect padding

# delete all reservations and rentals

In [2]:
db = SessionLocal()

date_from = date.today()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation).\
        filter(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date))
    items = db.query(models.CostumeItem).all()
    #db.query(models.Reservation).delete()
    for item in items:
        item.reservation_id = None
        item.rental_id = None
    db.commit() #uncomment it when u really want to deleta all reservations
    
    db.query(models.CostumeRental).delete()
    db.query(models.Reservation).delete()
    
    db.commit()
finally:
    db.close()